In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
import sys
import time
import html
import pathlib
from urllib.parse import urljoin, urlparse, parse_qs

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter, Retry

BASE_URL = "https://oku.korea.ac.kr/oku/cms/FR_CON/index.do?MENU_ID=820"
DOWNLOAD_DIR = pathlib.Path("korea_univ_guides")
TIMEOUT = 30
MAX_WORKERS = 6  # 병렬 다운로드 수 (requests만 사용 -> 간단히 순차로도 충분)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) PythonRequestsDownloader/1.0",
    "Referer": BASE_URL,
    "Accept": "*/*",
}

def make_session():
    s = requests.Session()
    retries = Retry(
        total=5,
        connect=5,
        read=5,
        backoff_factor=0.6,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD", "OPTIONS"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://", HTTPAdapter(max_retries=retries))
    s.headers.update(HEADERS)
    return s

def safe_filename(name: str, fallback: str = "file"):
    name = html.unescape(name).strip()
    # 줄바꿈/탭 제거
    name = re.sub(r"[\r\n\t]+", " ", name)
    # 너무 긴 이름 커팅
    name = name[:180]
    # 파일 시스템에 위험한 문자 제거 (한글/영문/숫자/공백/._-()만 허용)
    name = re.sub(r"[^0-9A-Za-z가-힣\.\-_\(\) ]+", "_", name)
    name = name.strip(" .")
    return name or fallback

def parse_content_disposition(cd_header: str):
    """
    Content-Disposition 예: attachment; filename="문과대학_국어국문학과_가이드.pdf"
    혹은 filename*=UTF-8''... 형태
    """
    if not cd_header:
        return None
    # RFC 5987 filename* 우선
    m = re.search(r"filename\*\s*=\s*UTF-8''([^;]+)", cd_header, flags=re.I)
    if m:
        try:
            from urllib.parse import unquote
            return unquote(m.group(1))
        except Exception:
            pass
    m = re.search(r'filename\s*=\s*"([^"]+)"', cd_header, flags=re.I)
    if m:
        return m.group(1)
    m = re.search(r"filename\s*=\s*([^;]+)", cd_header, flags=re.I)
    if m:
        return m.group(1).strip()
    return None

def extract_department_label(a_tag):
    """
    a_tag(“PDF 다운”) 주변에서 학과/전공명을 추정.
    HTML 구조상 '  * 학과명' 텍스트 라인이 있고 그 뒤에 [동영상] [PDF 다운] [홈페이지]가 이어짐.
    → a_tag의 부모 블록에서 직전 텍스트 노드를 찾아본다.
    """
    # 형제/부모를 거슬러 올라가며 최근에 등장한 학과명 라인을 찾는다.
    # 단순하지만 실무에 충분한 휴리스틱.
    label = None
    # 1) 바로 이전의 텍스트 노드
    prev = a_tag.find_previous(string=True)
    if prev:
        t = prev.strip()
        # '학과/학부/부' 등의 키워드가 있는 경우만 사용
        if any(kw in t for kw in ("학과", "학부", "부", "과")) and len(t) <= 30:
            label = t

    # 2) 못 찾으면 더 넓게: 이전에 나온 굵은 제목 등의 텍스트
    if not label:
        parent_text = a_tag.find_parent().get_text(" ", strip=True)
        # 새 줄 또는 불릿 앞 부분을 추정
        # 예: "국어국문학과 동영상 PDF 다운 홈페이지"
        m = re.match(r"^([^\s]+?(학과|학부|부))\b", parent_text)
        if m:
            label = m.group(1)

    return label

def discover_pdf_links(session):
    print("[*] 페이지 가져오는 중:", BASE_URL)
    r = session.get(BASE_URL, timeout=TIMEOUT)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    results = []
    college_idx = 0

    # 단과대학 단위 블록 찾기
    for block in soup.select("div.college_info"):
        # 단과대학 이름
        college_name_tag = block.find("strong")
        if not college_name_tag:
            continue
        college_idx += 1
        college_name = college_name_tag.get_text(strip=True)

        # 학과 링크들
        dept_links = block.find_all("a", string="PDF 다운")
        dept_idx = 0
        for a in dept_links:
            dept_idx += 1
            href = a.get("href")
            if not href or "FileDown.do" not in href:
                continue

            # 학과명 추출
            parent_text = a.find_parent().get_text(" ", strip=True)
            # "화공생명공학과 PDF 다운" → "화공생명공학과"
            dept_name = parent_text.replace("PDF 다운", "").strip()
            dept_name = dept_name.split()[0]

            results.append({
                "url": urljoin(BASE_URL, href),
                "college_idx": college_idx,
                "college_name": college_name,
                "dept_idx": dept_idx,
                "dept_name": dept_name
            })

    # 중복 제거 (url 기준)
    seen = set()
    uniq = []
    for x in results:
        if x["url"] not in seen:
            seen.add(x["url"])
            uniq.append(x)
    print(f"[*] 발견된 PDF 링크: {len(uniq)}개")
    return uniq

    print(f"[*] 발견된 PDF 링크: {len(results)}개")
    return results

def discover_pdf_links2(session: requests.Session):
    print("[*] 페이지 가져오는 중:", BASE_URL)
    r = session.get(BASE_URL, timeout=TIMEOUT)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    links = []
    for a in soup.find_all("a"):
        text = (a.get_text() or "").strip()
        href = a.get("href") or ""
        if ("PDF 다운" in text) or ("FileDown.do" in href):
            full_url = urljoin(BASE_URL, href)
            # 필터링: 실제 다운로드 엔드포인트만
            if "FileDown.do" in full_url:
                label = extract_department_label(a) or ""
                # 쿼리에서 MAJOR_SEQ, MAJOR_SUB_SEQ 추출
                q = parse_qs(urlparse(full_url).query)
                major_seq = q.get("MAJOR_SEQ", [""])[0]
                major_sub = q.get("MAJOR_SUB_SEQ", [""])[0]
                links.append({
                    "url": full_url,
                    "label": label,
                    "major_seq": major_seq,
                    "major_sub": major_sub
                })

    # 중복 제거 (url 기준)
    seen = set()
    uniq = []
    for x in links:
        if x["url"] not in seen:
            seen.add(x["url"])
            uniq.append(x)
    print(f"[*] 발견된 PDF 링크: {len(uniq)}개")
    return uniq

def ensure_pdf_suffix(name: str):
    return name if name.lower().endswith(".pdf") else (name + ".pdf")

def download_one(session: requests.Session, item: dict, outdir: pathlib.Path):
    url = item["url"]
    label = item["label"]
    seq = f"{item['major_seq']}_{item['major_sub']}".strip("_")


    # 우선 임시 파일명
    base_name = safe_filename(label, fallback=f"guide_{seq}")
    filename = ensure_pdf_suffix(base_name)
    dest = outdir / filename

    # 원하는 포맷: 06-공과대학-01.화공생명공학.pdf
    filename = f"{item['college_idx']:02d}-{safe_filename(item['college_name'])}-{item['dept_idx']:02d}.{safe_filename(item['dept_name'])}.pdf"
    dest = outdir / filename


    # 이미 있으면 스킵
    if dest.exists() and dest.stat().st_size > 0:
        print(f"[=] 존재함, 건너뜀: {dest.name}")
        return dest

    print(f"[>] 다운로드 시작: {dest.name}")
    with session.get(url, stream=True, timeout=TIMEOUT) as resp:
        resp.raise_for_status()
        # Content-Disposition에서 실제 파일명 가져오기
        cd = resp.headers.get("Content-Disposition", "")
        cd_name = parse_content_disposition(cd)
        if cd_name:
            cd_name = safe_filename(cd_name)
            if cd_name:
                dest = outdir / ensure_pdf_suffix(cd_name)

        tmp = dest.with_suffix(dest.suffix + ".part")
        tmp.parent.mkdir(parents=True, exist_ok=True)
        total = 0
        with open(tmp, "wb") as f:
            for chunk in resp.iter_content(chunk_size=1024 * 64):
                if chunk:
                    f.write(chunk)
                    total += len(chunk)
        os.replace(tmp, dest)
        print(f"[✔] 완료: {dest.name} ({total/1024:.1f} KB)")
        return dest

def main():
    DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
    session = make_session()
    links = discover_pdf_links(session)

    if not links:
        print("[!] PDF 링크를 찾지 못했습니다. 페이지 구조 변경 여부를 확인하세요.")
        sys.exit(1)

    # 순차 다운로드 (안정성 위주). 필요시 threading으로 바꿔도 됨.
    ok, fail = 0, 0
    for item in links:
        try:
            download_one(session, item, DOWNLOAD_DIR)
            ok += 1
            # 예의상 서버 부하 방지
            time.sleep(0.2)
        except Exception as e:
            fail += 1
            print(f"[x] 실패: {item['url']} -> {e}")

    print(f"\n[*] 완료 요약: 성공 {ok}건, 실패 {fail}건")
    print(f"[*] 저장 위치: {DOWNLOAD_DIR.resolve()}")

if __name__ == "__main__":
    main()

In [ ]:
%tb

In [ ]:
import urllib.parse
from pathlib import Path

def rename_pdf_files(folder: str):
    folder_path = Path(folder)
    for file in folder_path.glob("*.pdf"):
        name = file.stem  # 확장자 제외
        # "_EC_8B_AC" 같은 패턴을 "%EC%8B%AC"으로 변환
        encoded = re.sub(r"_([0-9A-Fa-f]{2})", r"%\1", name)
        try:
            decoded = urllib.parse.unquote(encoded)
        except Exception:
            print(f"[x] 디코딩 실패: {file.name}")
            continue

        # 새로운 파일명 (원래 있던 접두 숫자 유지)
        new_name = decoded + file.suffix
        new_path = file.with_name(new_name)

        # 같은 이름 파일이 이미 있으면 건너뜀
        if new_path.exists():
            print(f"[=] 이미 존재: {new_path.name}, 건너뜀")
            continue

        file.rename(new_path)
        print(f"[✔] {file.name} -> {new_path.name}")

rename_pdf_files("./korea_univ_guides")        